In [1]:
import sys

!which {sys.executable}

/ut3/jerome/miniconda3/envs/wakai/bin/python


In [2]:
import sys
sys.path.insert(0, '../code')

# Kounkel+2020 catalog

In [3]:
from catalog import CatalogDownloader

#no proper motion and radial velocity for members
cd = CatalogDownloader(catalog_name="Kounkel2020", catalog_type="cluster")
tabs = cd.get_tables_from_vizier(row_limit=-1)

Data url: https://vizier.u-strasbg.fr/viz-bin/VizieR?-source=J/AJ/160/279
{'J/AJ/160/279/table1': 'Clustered sources',
 'J/AJ/160/279/table2': 'Structure parameters',
 'J/AJ/160/279/tablea1': 'Parameters of clusters from Cantat-Gaudin+, 2018, '
                         'J/A+A/618/A93'}


In [4]:
df = tabs[0].to_pandas()
df_mem = tabs[2].to_pandas()
df_mem.head()

,Gaia,RAICRS,DEICRS,Group
0,2172342682494385024,320.572204,52.077340,1
1,2170224782576556672,315.652029,52.200351,1
2,2168760576695182208,315.716599,50.228894,1
3,2168760439256230528,315.763555,50.220269,1
4,2170222343035098880,315.580602,52.079478,1


In [9]:
from catalog import CatalogDownloader

#no proper motion and radial velocity for members
cd2 = CatalogDownloader(catalog_name="CantatGaudin2020", catalog_type="cluster")
tabs2 = cd2.get_tables_from_vizier(row_limit=-1)

Data url: https://vizier.u-strasbg.fr/viz-bin/VizieR?-source=J/A+A/633/A99
{'J/A+A/633/A99/members': '*Members',
 'J/A+A/633/A99/table1': 'Mean parameters of the identified clusters'}


In [23]:
df_mem2 = tabs2[1].to_pandas()

In [24]:
#replace name to be compatible with other catalogs' naming formats
df_mem2 = df_mem2.rename({"Cluster": "Name",
                        "Source": "Gaia",
                        "Proba": "Prob",
                        "RA_ICRS": "ra",
                        "DE_ICRS": "dec",
                        "pmRA": "pmra",
                        "pmDE": "pmdec",
                        "Plx": "parallax",
                        "RV": "radial_velocity"
                        }, axis=1)
df_mem2.head()

,ra,dec,Gaia,parallax,pmra,pmdec,radial_velocity,Gmag,BP-RP,Prob,Name,_RA.icrs,_DE.icrs
0,52.756558,34.689531,221090730122510976,1.3799,-1.601,-1.205,NaN,13.058100,0.9138,0.8,ASCC_10,52.756566,34.689536
1,52.730263,34.341461,221065132117493376,1.6550,-2.455,-1.998,NaN,14.775700,1.2807,0.6,ASCC_10,52.730275,34.341469
2,52.684345,34.003600,220988819138630016,1.5876,-2.437,-0.949,NaN,10.883200,0.7241,0.1,ASCC_10,52.684358,34.003604
3,52.676106,34.733687,221094367958697088,1.5586,-1.532,-1.558,NaN,16.879801,1.8380,0.7,ASCC_10,52.676114,34.733694
4,52.712100,34.578004,221083888238423552,1.4173,-1.403,-1.242,NaN,17.980000,2.2854,0.6,ASCC_10,52.712107,34.578009


# crossmatch Kounkel+20 and Cantat-Gaudin+20 for Gaia parameters

In [56]:
import pandas as pd

# df_mem_merged = pd.merge(df_mem, df_mem2, on='Gaia', how='left').dropna(subset=['ra','dec'])
df_mem_merged = pd.merge(df_mem, df_mem2, on='Gaia')
df_mem_merged.head()

,Gaia,RAICRS,DEICRS,Group,ra,dec,parallax,pmra,pmdec,radial_velocity,Gmag,BP-RP,Prob,Name,_RA.icrs,_DE.icrs
0,2168746901519321472,316.014106,50.245685,1,316.014111,50.245689,1.4210,1.151,-3.461,NaN,17.050699,3.2144,1.0,LDN_988e,316.014103,50.245704
1,2168758652549842944,316.008063,50.245369,1,316.008068,50.245374,1.6213,1.254,-3.290,NaN,17.033501,3.2971,1.0,LDN_988e,316.008060,50.245388
2,2168746867159585792,316.036612,50.242515,1,316.036616,50.242520,1.6660,1.399,-3.252,NaN,17.203800,3.9017,1.0,LDN_988e,316.036607,50.242534
3,2168746764080380288,316.067620,50.226425,1,316.067625,50.226430,1.6182,1.743,-3.512,NaN,17.585899,3.0895,1.0,LDN_988e,316.067613,50.226445
4,2168758274592720128,315.995854,50.241491,1,315.995859,50.241496,1.7025,1.044,-3.218,NaN,16.923000,2.8984,1.0,LDN_988e,315.995852,50.241510


In [57]:
df_mem_merged.shape

(113861, 16)

In [29]:
from catalog import get_tois

tois = get_tois(clobber=False)
tois.tail()

,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments,ra_deg,dec_deg,GaiaDR3_exofop,GaiaDR3_vizier
6140,201175570,7041.01,TIC 201175570.01,3,4,4,3,1,4,4,...,NaN,"68,69",2024-08-28,2024-08-28,2024-09-18 00:00:00,evolved host; low SNR; long duration,357.802167,-50.869861,6.522218e+18,6.522218e+18
6141,345724317,7042.01,NaN,2,4,2,2,1,4,4,...,0.184685,"25,52,79",2024-08-28,2024-08-28,2024-09-08 12:03:03,slight depth aperture correlation; weak BLS; l...,249.910167,20.743061,4.564173e+18,4.564173e+18
6142,57299130,7043.01,NaN,3,4,3,3,1,4,4,...,0.109921,"24,25,51,52,78,79",2024-08-28,2024-08-28,2024-09-08 12:03:02,some events on momentum dumps which cause appa...,251.806167,37.885958,1.352263e+18,1.352263e+18
6143,263723967,7044.01,NaN,3,4,3,3,1,4,4,...,0.136806,"25,52,79",2024-08-28,2024-08-28,2024-09-08 12:03:02,maximum period between events in each year is ...,250.414750,15.200531,4.462612e+18,4.462612e+18
6144,21720215,7045.01,TIC 21720215.01,3,4,3,3,1,4,4,...,0.192469,"25,79",2024-08-28,2024-08-28,2024-09-08 12:03:00,two stars in pixel; somewhat V-shaped,256.054042,31.565369,1.310410e+18,1.310410e+18


In [30]:
tois.shape

(6145, 66)

In [31]:
tois[tois.Comments.apply(lambda x: 'young' in str(x))]

,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments,ra_deg,dec_deg,GaiaDR3_exofop,GaiaDR3_vizier
732,146520535,942.01,NaN,3,5,5,3,4,4,4,...,0.097250,5,2019-07-24,2020-05-20,2022-12-14 12:09:24,Probable young K dwarf. Transit is believable ...,76.649625,-20.245614,2.974907e+18,2.974907e+18
2894,330135981,3604.01,NaN,3,4,4,3,2,4,4,...,NaN,"16,17,56,57",2021-06-23,2024-09-06,2024-09-06 00:00:00,V-shaped; found in faint-star QLP search; no s...,333.501042,56.918981,2.198435e+18,2.198435e+18
4887,308300731,5743.01,NaN,3,4,3,3,4,4,4,...,0.109025,"26,53,79,80",2022-09-01,2024-08-25,2024-08-31 12:14:55,radius possibly underestimated; potentially yo...,267.904333,24.427456,4.581034e+18,4.581034e+18
5078,236785891,5956.01,TIC 236785891.01,3,4,4,3,2,4,4,...,0.021842,"56,57",2022-11-16,2023-02-03,2024-09-01 00:00:00,V-shaped; large; young host star; ~5 sigma odd...,283.317250,59.817658,2.155491e+18,2.155491e+18
5365,435878153,6258.01,NaN,2,4,4,2,4,4,4,...,0.020209,"17,57",2023-04-06,2023-04-06,2024-02-23 00:00:00,low SNR; asymmetric transit shape; flaring pos...,13.085375,20.938875,2.789799e+18,2.789799e+18
5782,253864764,6678.01,NaN,1,4,1,1,3,4,4,...,0.092773,67,2023-09-21,2023-09-21,2024-09-12 00:00:00,rotational modulation at 7-8 days may indicate...,283.536292,-36.656858,6.731034e+18,6.731034e+18


# cross-match TOI with catalog

In [32]:
import pandas as pd

if True:
    #from astroquery.xmatch
    matches = pd.read_csv('../data/TOIs_GaiaDR3_xmatch.csv')
if False:
    #from vizier query/exofop
    matches = pd.read_csv('../data/TOIs_with_Gaiaid.csv')
    matches['GaiaDR3'] = matches['GaiaDR3_vizier']
matches.head()

,angDist,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,...,E(BP-RP),b_E(BP-RP),B_E(BP-RP),Lib,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000,GaiaDR3
0,0.031583,231663901,101.01,NaN,5,5,5,5,5,5,...,0.0088,0.0015,0.0438,MARCS,318.736916,-55.871794,0.183824,0.186203,-0.0885,6462994429708755072
1,0.029564,149603524,102.01,NaN,5,5,5,5,5,5,...,0.0005,0.0001,0.0013,MARCS,87.139972,-63.988441,0.270854,0.243980,-0.2165,4756649415309914240
2,0.035823,336732616,103.01,NaN,5,5,5,5,5,5,...,0.0180,0.0144,0.0210,MARCS,312.457439,-24.428761,0.282852,0.230957,0.4094,6805886373600546176
3,0.016955,231670397,104.01,NaN,5,5,5,5,5,5,...,NaN,NaN,NaN,NaN,319.949611,-58.148887,0.213587,0.200585,-0.1912,6453566701615683456
4,0.024290,144065872,105.01,NaN,5,5,5,5,5,5,...,0.0174,0.0136,0.0234,MARCS,337.457231,-48.003069,0.180076,0.230102,0.1517,6518399301667782016


In [33]:
matches.shape

(7304, 205)

In [34]:
final = pd.merge(df_mem_merged, matches, left_on='Gaia', right_on='GaiaDR3')
final.head()

,Gaia,RAICRS,DEICRS,Group,ra,dec,parallax,pmra,pmdec,radial_velocity,...,E(BP-RP),b_E(BP-RP),B_E(BP-RP),Lib,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000,GaiaDR3
0,3220568964775337216,81.312672,-0.770745,13,81.312676,-0.770748,2.7452,0.953,-0.742,NaN,...,0.5312,0.5248,0.5372,MARCS,81.312671,-0.770747,1.321691,0.947552,-0.0195,3220568964775337216
1,5519619186857962112,123.108270,-46.109135,22,123.108274,-46.109140,2.8585,-5.990,9.892,NaN,...,0.1346,0.1309,0.1382,A,123.108313,-46.109183,0.279479,0.272169,0.0143,5519619186857962112
2,2014335027560174976,343.233208,59.850947,32,343.233217,59.850951,2.4396,4.326,-2.168,NaN,...,0.2932,0.2916,0.2950,A,343.233179,59.850961,0.263616,0.265080,-0.2499,2014335027560174976
3,5295126507633392128,115.639046,-58.623267,79,115.639048,-58.623272,2.6501,-4.994,7.688,NaN,...,0.1807,0.1787,0.1827,A,115.639088,-58.623305,0.245183,0.209736,0.1324,5295126507633392128
4,5251470948229949568,157.037272,-64.505206,92,157.037281,-64.505211,6.9893,-18.017,11.307,17.445309,...,0.0158,0.0104,0.0202,MARCS,157.037460,-64.505260,0.231326,0.219843,0.0296,5251470948229949568


In [46]:
final2 = pd.merge(final, df[['Group','OName','logAge']], on='Group')
final.head()

,Gaia,RAICRS,DEICRS,Group,ra,dec,parallax,pmra,pmdec,radial_velocity,...,E(BP-RP),b_E(BP-RP),B_E(BP-RP),Lib,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000,GaiaDR3
0,3220568964775337216,81.312672,-0.770745,13,81.312676,-0.770748,2.7452,0.953,-0.742,NaN,...,0.5312,0.5248,0.5372,MARCS,81.312671,-0.770747,1.321691,0.947552,-0.0195,3220568964775337216
1,5519619186857962112,123.108270,-46.109135,22,123.108274,-46.109140,2.8585,-5.990,9.892,NaN,...,0.1346,0.1309,0.1382,A,123.108313,-46.109183,0.279479,0.272169,0.0143,5519619186857962112
2,2014335027560174976,343.233208,59.850947,32,343.233217,59.850951,2.4396,4.326,-2.168,NaN,...,0.2932,0.2916,0.2950,A,343.233179,59.850961,0.263616,0.265080,-0.2499,2014335027560174976
3,5295126507633392128,115.639046,-58.623267,79,115.639048,-58.623272,2.6501,-4.994,7.688,NaN,...,0.1807,0.1787,0.1827,A,115.639088,-58.623305,0.245183,0.209736,0.1324,5295126507633392128
4,5251470948229949568,157.037272,-64.505206,92,157.037281,-64.505211,6.9893,-18.017,11.307,17.445309,...,0.0158,0.0104,0.0202,MARCS,157.037460,-64.505260,0.231326,0.219843,0.0296,5251470948229949568


In [47]:
final2.shape

(21, 223)

In [48]:
final2.sort_values(by='TOI', ascending=False).to_csv(f'../data/{cd.catalog_name}/TOIs_in_{cd.catalog_name}.csv', index=False)